# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:

import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import logging
import os
import csv

from matplotlib import pyplot as plt

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core import Environment, ScriptRunConfig
from azureml.core.dataset import Dataset
from azureml.core.model import Model
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.sklearn import SKLearn
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)


In [ ]:
ws = Workspace.from_config()
experiment_name = 'loan_prediction'

experiment=Experiment(workspace=ws, name=experiment_name)


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

In [ ]:
# compute cluster

amlcompute_cluster_name = "cpu-clusters"

try:
    remote_run_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=4)
    remote_run_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

remote_run_compute.wait_for_completion(show_output=True , min_node_count = 1, timeout_in_minutes = 2)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
 # Create TabularDataset using TabularDatasetFactory
    # Data is located at:

data_path = "https://raw.githubusercontent.com/Tekhunt/ML-Project/master/loands.csv"

data = TabularDatasetFactory.from_delimited_files(path= data_path)
data.to_pandas_dataframe().head()

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
        "--C": choice(0.001, 0.01, 0.1, 1, 10, 100, 1000),
        "--max_iter": choice(50, 100, 150, 200, 250, 500)
    }
)
#experiment_folder = 'train_file'
#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory = './',
                 entry_script = 'train_script.py',
                 compute_target = remote_run_compute)

hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                                hyperparameter_sampling=param_sampling,
                                policy = policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=4,
                                max_concurrent_runs=4)

In [ ]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
# Visualize hyperparameter tuning runs
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()

In [ ]:
print(best_run.get_details()['runId'])

In [ ]:
print(best_run.get_details()['runDefinition']['arguments'])

In [ ]:

# Model Metrics
print(best_run.get_metrics())

In [ ]:
best_run.get_details()

In [ ]:

print(best_run.get_file_names())

In [ ]:
# Save the best model 
best_run.download_file('/outputs/model.joblib', 'hyperdrive_model.joblib')

In [ ]:
# Register the best model
model = best_run.register_model(model_name='hyperdrive_model',
                                model_path='outputs/model.joblib',
                                model_framework=Model.Framework.SCIKITLEARN)
print(model)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
service_name = 'hyperdrive-service'
service = Model.deploy(ws, service_name, [model])
service.wait_for_deployment(show_output=True)

In [ ]:
print(service.state)

In [ ]:
###hhhhhh

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:

x_test=pd.read_csv("test_data_cleaned.csv")
x_test = x_test.drop(['Customer Id', 'Building_Painted', 'Building_Fenced','Garden','Settlement'], axis=1)

In [ ]:
service

In [ ]:

#Running predictions on sample data
import requests
import json
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
#primary,secondary = aci_service.get_keys()
#key = primary

# Convert to JSON string
x_test_json = x_test.to_json(orient='records')
data = "{\"data\": " + x_test_json +"}"

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, data, headers=headers)
print(resp.text)
#resp = requests.post(aci_service.scoring_uri, data, headers=headers)

y_pred = json.loads(json.loads(resp.text))['result']

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:

service.delete()
aml_compute.delete()